In [1]:
import numpy as np
import pandas as pd
import math


In [3]:
# Dataset
data = pd.DataFrame({
    "Gender": ["M", "F", "M", "F", "M"],
    "Age": [38, 52, 45, 29, 61],
    "Income": [420000, 360000, 780000, 300000, 500000],
    "Smoking": [0, 5, 0, 12, 8],
    "Illness": [-1, +1, +1, +1, +1]   # Yes=+1, No=-1
})

# Initial weights
n = len(data)
data["weight"] = 1 / n

data


,Gender,Age,Income,Smoking,Illness,weight
0,M,38,420000,0,-1,0.2
1,F,52,360000,5,1,0.2
2,M,45,780000,0,1,0.2
3,F,29,300000,12,1,0.2
4,M,61,500000,8,1,0.2


In [5]:
def stump_smoking(row):
    return 1 if row["Smoking"] >= 1 else -1

def stump_age(row):
    return 1 if row["Age"] >= 45 else -1


In [7]:
def weighted_error(data, stump):
    error = 0
    for i, row in data.iterrows():
        if stump(row) != row["Illness"]:
            error += row["weight"]
    return error

def compute_alpha(error):
    return 0.5 * math.log((1 - error) / error)

def update_weights(data, stump, alpha):
    for i, row in data.iterrows():
        prediction = stump(row)
        data.at[i, "weight"] *= math.exp(-alpha * row["Illness"] * prediction)

    # Normalize
    data["weight"] = data["weight"] / data["weight"].sum()


In [19]:
#🔹 ROUND-1 (Smoking Stump)

In [9]:
error1 = weighted_error(data, stump_smoking)
alpha1 = compute_alpha(error1)

update_weights(data, stump_smoking, alpha1)

print("Round 1")
print("Error:", error1)
print("Alpha:", alpha1)
data


Round 1
Error: 0.2
Alpha: 0.6931471805599453


,Gender,Age,Income,Smoking,Illness,weight
0,M,38,420000,0,-1,0.125
1,F,52,360000,5,1,0.125
2,M,45,780000,0,1,0.500
3,F,29,300000,12,1,0.125
4,M,61,500000,8,1,0.125


In [21]:
#🔹 ROUND-2 (Age Stump)

In [11]:
error2 = weighted_error(data, stump_age)
alpha2 = compute_alpha(error2)

update_weights(data, stump_age, alpha2)

print("Round 2")
print("Error:", error2)
print("Alpha:", alpha2)
data


Round 2
Error: 0.125
Alpha: 0.9729550745276566


,Gender,Age,Income,Smoking,Illness,weight
0,M,38,420000,0,-1,0.071429
1,F,52,360000,5,1,0.071429
2,M,45,780000,0,1,0.285714
3,F,29,300000,12,1,0.500000
4,M,61,500000,8,1,0.071429


In [23]:
#🔹 ROUND-3 (Smoking Again)

In [13]:
error3 = weighted_error(data, stump_smoking)
alpha3 = compute_alpha(error3)

print("Round 3")
print("Error:", error3)
print("Alpha:", alpha3)


Round 3
Error: 0.28571428571428575
Alpha: 0.45814536593707733


In [15]:
def final_prediction(row):
    score = (
        alpha1 * stump_smoking(row) +
        alpha2 * stump_age(row) +
        alpha3 * stump_smoking(row)
    )
    return 1 if score > 0 else -1

data["Final_Pred"] = data.apply(final_prediction, axis=1)
data


,Gender,Age,Income,Smoking,Illness,weight,Final_Pred
0,M,38,420000,0,-1,0.071429,-1
1,F,52,360000,5,1,0.071429,1
2,M,45,780000,0,1,0.285714,-1
3,F,29,300000,12,1,0.500000,1
4,M,61,500000,8,1,0.071429,1


In [17]:
accuracy = (data["Final_Pred"] == data["Illness"]).mean()
print("Final Accuracy:", accuracy)


Final Accuracy: 0.8
